# 10 - Power spectra

In this tutorial we will compare simulated data to analytic dispersion relations.

The Struphy run generating the data for this tutorial can be launched via:

    $ struphy test tutorials -n 4
    
All data generated in this way is stored in the Struphy installation path (obtained from ``struphy -p``) under ``io/out/tutorial_04``.

## Maxwell

In [ ]:
import os
import struphy

path_out = os.path.join(struphy.__path__[0], 'io/out/', 'tutorial_04a/')

os.listdir(path_out)

Let us get some meta data on the simulation used in this notebook:

In [ ]:
with open(os.path.join(path_out, 'meta.txt')) as file:
    print(file.read())

Here, the model [Maxwell](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.toy.Maxwell) was simulated on 2 MPI processes, which can be confirmed by looking at the raw `.hdf5` data:

In [ ]:
os.listdir(os.path.join(path_out, 'data'))

Let us look at the parameter file of the simulation:

In [ ]:
params_path = os.path.join(struphy.__path__[0], 'io/inp', 'tutorials', 'params_04a.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
    
parameters

The simulation has resolution only in the third ($\eta_3$-) direction and was performed with periodic boundary conditions.
The geometry was a [Cuboid](https://struphy.pages.mpcdf.de/struphy/sections/domains.html#struphy.geometry.domains.Cuboid) with domain length $L_z=20$ in the third direction.

The first and second component of the electric field `e1` have been initialized with `noise` of amplitude 0.1 varying in the third direction.

The raw data has already been post-processed (presence of the folder `post_processing/`). Let us look at the resulting data:

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing'))

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'fields_data'))

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'fields_data', 'em_fields'))

Our aim is to perform a Fourier analysis of the data and to see whether the data contains the dispersion relation $\omega(k_z)$ of light waves. For this, we perform a 2d discrete FFT on the data points in the $(t, z)$-plane. The FFT maps $E(t, z) \to \hat E(\omega, k)$, where $|\hat E(\omega, k)|^2$ is called the **power spectrum** of $E$.

First, we load the data and the grids:

In [ ]:
import pickle

# load data dicts for e_field
with open(os.path.join(path_out, 'post_processing', 'fields_data', 'em_fields', 'e_field_log.bin'), 'rb') as handle:
    point_data_log = pickle.load(handle)

# load grid
with open(os.path.join(path_out, 'post_processing', 'fields_data', 'grids_log.bin'), 'rb') as handle:
    grids_log = pickle.load(handle)

with open(os.path.join(path_out, 'post_processing', 'fields_data', 'grids_phy.bin'), 'rb') as handle:
    grids_phy = pickle.load(handle)

The computation of the power spectrum has been automatized in Struphy:

In [ ]:

from struphy.diagnostics.diagn_tools import power_spectrum_2d

# fft in (t, z) of first component of e_field on physical grid
power_spectrum_2d(point_data_log,
                  'e1',
                  'Maxwell',
                  grids_log,
                  grids_mapped=grids_phy,
                  component=0,
                  slice_at=[0, 0, None],
                  do_plot=True,
                  disp_name='Maxwell1D')

## Linear MHD

In [ ]:
path_out = os.path.join(struphy.__path__[0], 'io/out/', 'tutorial_04b/')

os.listdir(path_out)

Let us get some meta data on the simulation used in this notebook:

In [ ]:
with open(os.path.join(path_out, 'meta.txt')) as file:
    print(file.read())

Here, the model [LinearMHD](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.fluid.LinearMHD) was simulated on 2 MPI processes, which can be confirmed by looking at the raw `.hdf5` data:

In [ ]:
os.listdir(os.path.join(path_out, 'data'))

Let us look at the parameter file of the simulation:

In [ ]:
params_path = os.path.join(struphy.__path__[0], 'io/inp', 'tutorials', 'params_04b.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
    
parameters

The simulation has resolution only in the third ($\eta_3$-) direction and was performed with periodic boundary conditions.
The geometry was a [Cuboid](https://struphy.pages.mpcdf.de/struphy/sections/domains.html#struphy.geometry.domains.Cuboid) with domain length $L_z=60$ in the third direction.

All comonents of the MHD velocity `u2` have been initialized with `noise` of amplitude 0.1 varying in the third direction.

The MHD equilibirum was a [HomogenSlab](https://struphy.pages.mpcdf.de/struphy/sections/mhd_equils.html#struphy.fields_background.mhd_equil.equils.HomogenSlab). We have to extract some parameters for the computation of the analytic dispersion relation:

In [ ]:
B0x = parameters['mhd_equilibrium']['HomogenSlab']['B0x']
B0y = parameters['mhd_equilibrium']['HomogenSlab']['B0y']
B0z = parameters['mhd_equilibrium']['HomogenSlab']['B0z']

p0 = parameters['mhd_equilibrium']['HomogenSlab']['beta'] * (B0x**2 + B0y**2 + B0z**2) / 2
n0 = parameters['mhd_equilibrium']['HomogenSlab']['n0']

disp_params = {'B0x': B0x,
               'B0y': B0y,
               'B0z': B0z,
               'p0': p0,
               'n0': n0,
               'gamma': 5/3}


The raw data has already been post-processed (presence of the folder `post_processing/`). Let us look at the resulting data:

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing'))

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'fields_data'))

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'fields_data', 'mhd'))

Our aim is to perform a Fourier analysis of the data and to see whether the data contains the dispersion relation $\omega(k_z)$ of ideal MHD in the slab. For this, we perform two 2d discrete FFT on the data points in the $(t, z)$-plane:

* velocity $U(t, z) \to \hat U(\omega, k)$
* pressure $p(t, z) \to \hat p(\omega, k)$

First, we load the data and the grids:

In [ ]:
import pickle

# load data dicts for velocity and pressure
with open(os.path.join(path_out, 'post_processing', 'fields_data', 'mhd', 'velocity_log.bin'), 'rb') as handle:
    u2_log = pickle.load(handle)
    
with open(os.path.join(path_out, 'post_processing', 'fields_data', 'mhd', 'pressure_log.bin'), 'rb') as handle:
    p3_log = pickle.load(handle)

# load grid
with open(os.path.join(path_out, 'post_processing', 'fields_data', 'grids_log.bin'), 'rb') as handle:
    grids_log = pickle.load(handle)

with open(os.path.join(path_out, 'post_processing', 'fields_data', 'grids_phy.bin'), 'rb') as handle:
    grids_phy = pickle.load(handle)

The power spectrum of the velocity should yield the shear Afvén wave:

In [ ]:
from struphy.diagnostics.diagn_tools import power_spectrum_2d

# fft in (t, z) of first component of e_field on physical grid
power_spectrum_2d(u2_log,
                  'u2',
                  'LinearMHD',
                  grids_log,
                  grids_mapped=grids_phy,
                  component=0,
                  slice_at=[0, 0, None],
                  do_plot=True,
                  disp_name='MHDhomogenSlab',
                  disp_params=disp_params)

The power spectrum of the pressure should yield the slow and fast magnetosonic waves, respectively:

In [ ]:
power_spectrum_2d(p3_log,
                  'p3',
                  'LinearMHD',
                  grids_log,
                  grids_mapped=grids_phy,
                  component=0,
                  slice_at=[0, 0, None],
                  do_plot=True,
                  disp_name='MHDhomogenSlab',
                  disp_params=disp_params)